In [1]:
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import h5py

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing import sequence 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Attention
from tensorflow.keras.datasets import imdb

In [6]:
(x_train,y_train), (x_test, y_test) = imdb.load_data(num_words=15000)

In [7]:
x_train = sequence.pad_sequences(x_train, maxlen = 150)
x_test = sequence.pad_sequences(x_test, maxlen = 150)

In [8]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [19]:
main_input = Input(shape=( 150 ), name='main_input')

In [20]:
x1 = Embedding(output_dim=512, input_dim=15000, input_length=150)(main_input)
x2 = Embedding(output_dim=512, input_dim=15000, input_length=150)(main_input)

In [21]:
query_value_attention_seq = tf.keras.layers.Attention()(
    [x1, x2])

# Reduce over the sequence axis to produce encodings of shape
# [batch_size, filters].
query_encoding = tf.keras.layers.GlobalAveragePooling1D()(
    x1)
query_value_attention = tf.keras.layers.GlobalAveragePooling1D()(
    query_value_attention_seq)

#Concatenate query and document encodings to produce a DNN input layer.
input_layer = tf.keras.layers.Concatenate()(
    [query_encoding, query_value_attention])

In [22]:
x = Dense(64, activation='relu')(input_layer)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
#得到输出的张量
# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [23]:
model = Model(inputs=main_input, outputs=main_output)
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics = ["accuracy"])

In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 150, 512)     7680000     main_input[0][0]                 
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 150, 512)     7680000     main_input[0][0]                 
__________________________________________________________________________________________________
attention_1 (Attention)         (None, 150, 512)     0           embedding_2[0][0]                
                                                                 embedding_3[0][0]          

In [25]:
model.fit(x_train,y_train ,epochs=1, batch_size=32)

Train on 25000 samples
25000/25000 [==============================] - 572s 23ms/sample - loss: 0.3616 - accuracy: 0.8389


In [26]:
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 53s 2ms/sample - loss: 0.3157 - accuracy: 0.8631


[0.31571699115276336, 0.86308]